In [1]:
import os
from pathlib import Path

# 1. MOVE TO ROOT
# Check if we are in 'research' and move up
if os.getcwd().endswith("research"):
    os.chdir("..")
print(f"Current Directory: {os.getcwd()}")

Current Directory: d:\OneDrive\Desktop\disaster_victim_detection


In [2]:
from dataclasses import dataclass
from pathlib import Path
from src.victimDetector.constants import *
from src.victimDetector.utils.common import read_yaml, create_directories

In [3]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    params_model_type: str

In [4]:
# 3. CONFIG MANAGER
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            params_model_type=self.params.yolo_params.model_type
        )

        return prepare_base_model_config

In [5]:
from ultralytics import YOLO
import shutil

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        # This downloads yolov8n.pt from Ultralytics servers
        print(f"Downloading base model: {self.config.params_model_type}")
        model = YOLO(self.config.params_model_type) 
        
        # Save it to our artifacts folder
        # YOLO download logic is a bit implicit, so we save explicitly
        model.save(self.config.base_model_path)
        print(f"Base model saved at: {self.config.base_model_path}")

In [6]:
# 5. PIPELINE EXECUTION
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
except Exception as e:
    raise e

[2026-01-17 00:08:28,914: INFO: common: YAML file loaded successfully: config\config.yaml]
[2026-01-17 00:08:28,917: INFO: common: YAML file loaded successfully: params.yaml]
[2026-01-17 00:08:28,918: INFO: common: created directory at: artifacts]
[2026-01-17 00:08:28,919: INFO: common: created directory at: artifacts/prepare_base_model]
Base model saved at: artifacts\prepare_base_model\base_model.pt
